In [1]:
import plotly.express as px
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from jupyter_dash import JupyterDash
from dash import Dash
import dash_core_components as dcc
import dash_html_components as html
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from dash.dependencies import Input, Output

In [5]:
import sys
sys.path.append("../src/")
from raingauge_manager import Raingauge

In [41]:
raingauge=Raingauge()
raingauge.set_db()
df_hourly=raingauge.get_precipitation(begin="2022-10-08 00:00:00", end="2022-10-10 23:59:59")
df_daily=raingauge.get_precipitation(begin="2022-10-01 00:00:00", end="2022-10-31 23:59:59", freq="D")

In [56]:
datetimes=[t.strftime("%Y%m%d") for t in df_daily.index]
app=JupyterDash(__name__)
app.layout=html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(id="date-button",options=[{"label":t,"value":t} for t in datetimes], value="20221014")
        ]),
    dcc.Graph(id="graph"),

    ])
])

@app.callback(
    Output("graph","figure"),
    Input("date-button","value")
)
def update_figure(date_button):
    raingauge=Raingauge()
    raingauge.set_db()
    df_hourly=raingauge.get_precipitation(begin="2022-10-08 00:00:00", end="2022-10-10 23:59:59")
    df_daily=raingauge.get_precipitation(begin="2022-10-01 00:00:00", end="2022-10-31 23:59:59", freq="D")
    
    fig=make_subplots(rows=2,cols=1,subplot_titles=["hourly","daily"])
    fig.add_trace(go.Bar(x=df_hourly.index, y=df_hourly["precip_mm"]),row=1,col=1 )
    fig.add_trace(go.Bar(x=df_daily.index, y=df_daily["precip_mm"]), row=2,col=1)
    fig.update_layout(
        height=700
    )
    
    del raingauge
    return fig

In [1]:
# if __name__=="__main__":
#     app.run_server(debug=True, mode="inline", port=8999,host="localhost")